# Web Scraping

In [ ]:
# NOTE: 爬 DOI

import json
import requests
# import pprint


api_key = '43a3b963318fd18b917378a5f89e75c6'
query = 'temperature'
base_url = 'https://api.elsevier.com/content/search/sciencedirect'

data = {"qs": query,
        "date": 2019,
        "volume": 1,  ### VOLUME 册
        "display": {
            "show": 100, # 展示多少篇文章
            "offset": 1}} # 第几篇文章

headers = {'x-els-apikey': '43a3b963318fd18b917378a5f89e75c6',
           'Content-Type': 'application/json',
           'Accept': 'application/json'}

def get_response(url, data, headers):
    response = requests.put(url, data=json.dumps(data), headers=headers) # 把本地要发出去的数据打包发到服务器
    response = response.text.replace('false', 'False').replace('true', 'True')
    try:
        response = eval(response)
    except:
        print(response)
    return response


response = get_response(base_url, data, headers)
dois=[]
if 'results' in response.keys():
    results = response['results']
    for result in results:
        if 'doi' in result:
            dois.append(result['doi'])
print(len(dois))

In [ ]:
dois = []

In [ ]:
# NOTE: 爬论文
import requests

with open('elsevier_dois.json') as f:
    dois = f.read().strip().split('\n')
    print(len(dois))

api_key = '43a3b963318fd18b917378a5f89e75c6'
with open('start.txt') as f: start = int(f.read().strip())

for i in range(start, len(dois)):
# for i in range(1802, 1803):
    try:
        with open(f'elsevier_papers/elsevier_{i}.xml', 'w', encoding='utf-8') as f:
            request_url = 'https://api.elsevier.com/content/article/doi/{}?apiKey={}&httpAccept=text%2Fxml'.format(dois[i], api_key)
            text = requests.get(request_url).text
            if text is not None:
                f.write(text)
                if i % 50 == 0: print(i, end = ' ')
                with open('start.txt', 'w') as f: f.write(str(i))
    except Exception:
        text = None
        continue


In [2]:
import requests

with open('dois.json') as f:
    dois = f.read().strip().split('\n')
    print(len(dois))

api_key = '43a3b963318fd18b917378a5f89e75c6'
with open('start.txt') as f: start = int(f.read().strip())

for i in range(start, len(dois)):
# for i in range(1802, 1803):
    try:
        with open(f'elsevier_papers/elsevier_{i}.xml', 'w', encoding='utf-8') as f:
            request_url = 'https://api.elsevier.com/content/article/doi/{}?apiKey={}&httpAccept=text%2Fxml'.format(dois[i], api_key)
            text = requests.get(request_url).text
            if text is not None:
                f.write(text)
                if i % 50 == 0: print(i, end = ' ')
                with open('start.txt', 'w') as f: f.write(str(i))
    except Exception:
        text = None
        continue

    

3273422


# Tokenizing

In [ ]:
from bs4 import BeautifulSoup

with open('elsevier_dois.json') as f:
    dois = f.read().strip().split('\n')
    print(len(dois))

# NOTE: 提取 DOI, abstract, title, 和 year
def extract(folder, xml): 
# def extract(paper):
    f = open(folder + '/' + xml, encoding = 'utf-8').read()
    doc = BeautifulSoup(f, 'xml')
    # print(doc)
    if doc.find('doi') is None: return
    p_doi = doc.find('doi').string
    p_abst = None
    p_title = None
    p_year = None
    
    if doc.find('title') != None and doc.find('title').string != None:
        temp = doc.find('title').string.split()
        p_title = ' '.join(temp)

    if doc.find('coverDisplayDate') != None and doc.find('coverDisplayDate').string != None:
        p_year = int(doc.find('coverDisplayDate').string.strip().split(' ')[-1][:4])
        # print(p_year)
    if doc.find('description') != None and doc.find('description').string != None:
        # p_abst = doc.find('description').string.strip()
        temp = doc.find('description').string.split()
        p_abst = ' '.join(temp)
        # print(p_abst)
    
    
    return (p_doi, p_abst, p_title, p_year)

In [2]:
import json
import os
import sys
sys.path.append('..')
from LimeSoup import (
    ACSSoup, 
    # AIPSoup,
    APSSoup, 
    ECSSoup, 
    ElsevierSoup, 
    IOPSoup, 
    NatureSoup, 
    RSCSoup, 
    SpringerSoup, 
    WileySoup,
)
from bs4 import BeautifulSoup

def findAllFiles(base):
    result = []
    for root, ds, fs in os.walk(base):
        # print(fs)
        result += fs
    return result

def find_content(dic):
    result = ''
    if isinstance(dic, str): 
        result += dic + ' '
        return result
    if isinstance(dic, list):
        for l in dic:
            result += find_content(l)
    if isinstance(dic, dict):
        result += find_content(dic['content'])
    return result
        


def rsc_extract(html):
    data = RSCSoup.parse(html)

    doi, title, abstract = '', '', ''

    year = BeautifulSoup(html, 'html').find('meta', {'name': 'DC.Date'}).get('content')[:4]

    if 'DOI' in data: doi = data['DOI']
    if 'Title' in data: title = data['Title']
    if 'Sections' in data: 
        result = ''
        for dic in data['Sections']:
            result += find_content(dic)
        abst = result

    return doi, abst, title, year

In [3]:
doi_absts = {}
index = 0
for fname in findAllFiles('rsc1'):
    with open(f'rsc1/{fname}', 'r', encoding = 'utf-8') as f:
        html = f.read()
    # print(fname)
    try:
        result = rsc_extract(html)
        index += 1
        temp = index
        if result is None: 
            continue
        else: doi, abst, title, year = result
        if year not in doi_absts: doi_absts[year] = [(doi, title, abst)]
        else: doi_absts[year].append((doi, title, abst))
        if index % 50 == 0: print(index, end = ' ')
    except Exception: continue

print(index)


50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500 2550 2600 2650 2700 2750 2800 2850 2900 2950 3000 3050 3100 3150 3200 3250 3300 3350 3400 3450 3500 3550 3600 3650 3700 3750 3800 3850 3900 3950 4000 4050 4100 4150 4200 4250 4300 4350 4400 4404


In [10]:
years = []
for year in doi_absts:
    years.append(year)
years.sort()
print(years)



['', '1906', '1922', '1956', '1959', '1961', '1962', '1963', '1964', '1965', '1970', '1971', '1983', '1985', '1986', '1988', '1991', '1992', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']


In [ ]:
# NOTE: 将论文装进字典里
doi_absts = {}
index = 0
while index < len(dois):
    name = f'elsevier_{index}.xml'
    result = extract('elsevier_papers', name)
    index += 1
    temp = index
    if result is None: 
        continue
    else: doi, abst, title, year = result
    if year not in doi_absts: doi_absts[year] = [(doi, title, abst)]
    else: doi_absts[year].append((doi, title, abst))
    if index % 300 == 0: print(index, end = ' ')

print(index)


In [ ]:
from pprint import pprint
print(len(doi_absts))
# pprint(doi_absts)

In [13]:
# NOTE: 创建数据库
import pymongo
client = pymongo.MongoClient('mongodb://localhost:27017')
db_papers = client['papers']

# NOTE: 初始化数据库
def papers_init(db_papers):
    # for col in db_papers.list_collection_names():
    #     de = db_papers[col]
    #     db_papers.drop_collection(de)

    # for year in range(2000, 2022):
    for year in years:
        if year == '': year = 'None'
        col = db_papers[str(year)]
        if year == 'None': year = ''
        for i in doi_absts[year]:
            doi, title, abst = i
            paper = {
                'doi': doi,
                'title': title,
                'abst': abst
            }
            col.insert_one(paper)

papers_init(db_papers)

In [11]:
# NOTE: 从数据库中读论文
year_papers = {}
# print(db_papers.list_collection_names())
for year in db_papers.list_collection_names():
    for paper in db_papers[year].find({}, {'_id': 0}):
        if year not in year_papers: year_papers[year] = [(paper['doi'], paper['title'], paper['abst'])]
        else: year_papers[year].append((paper['doi'], paper['title'], paper['abst']))
# print(year_papers)

In [12]:
# NOTE: 一共有多少单词
text = []
for year in year_papers:
    for paper in year_papers[year]:
        abst = paper[2]
        
        # print(doi_absts[doi])
        if abst != None : text += abst.strip().split(' ')
print(len(text))
print(len(year_papers))
# print(text)

17580778
22


In [14]:
text = []
for year in doi_absts:
    for paper in doi_absts[year]:
        abst = paper[2]
        
        # print(doi_absts[doi])
        if abst != None : text += abst.strip().split(' ')
print(len(text))
print(len(doi_absts))
# print(text)

26101351
46


In [ ]:
from chemdataextractor.doc import Document
import sys
sys.path.append('..')
from mat2vec.processing import MaterialsTextProcessor

processor = MaterialsTextProcessor()

year_chemicals = {}

for year, paper_lst in year_papers.items():
    # index = 0
    print(year)
    result = ''
    for tup in paper_lst:
        
        # if index % 100 == 0: print(index, end = ' ')
        # index += 1
        result += str(tup[1]) + " " + str(tup[2])
    
    print(len(result))
    exclude = {'a', 'an', 'is', 'are', ',', "=", ".", ",", "(", ")", "<", ">", "\"", "“", "”", "≥", "≤", "<nUm>", "been", "be", "are",
                 "which", "were", "where", "have", "important", "has", "can", "or", "we", "our",
                 "article", "paper", "show", "there", "if", "these", "could", "publication",
                 "while", "measured", "measure", "demonstrate", "investigate", "investigated",
                 "demonstrated", "when", "prepare", "prepared", "use", "used", "determine",
                 "determined", "find", "successfully", "newly", "present",
                 "reported", "report", "new", "characterize", "characterized", "experimental",
                 "result", "results", "showed", "shown", "such", "after",
                 "but", "this", "that", "via", "is", "was", "and", "using"}
    visited = set()
    # result = ' '.join(set(result.split(' ')))
    result = processor.process(str(result))[0]
    temp = ''
    for i in result:
        if i in visited or i in exclude: continue
        else:
            visited.add(i)
            temp += i + ' '
    
    print(len(temp))
        
    # print(temp)
    # print('aaa')
    
    doc = Document(temp)
    # print('aaa')
    for dic in doc.records.serialize():
        # print('a')
        if 'names' in dic:
            chemicals = set(dic['names'])
            if year not in year_chemicals: year_chemicals[year] = chemicals
            else: year_chemicals[year] = year_chemicals[year].union(chemicals)

print(len(year_chemicals))


In [15]:
import regex
import string
import unidecode
from os import path
from monty.fractions import gcd_float

from chemdataextractor.doc import Paragraph


from pymatgen.core.periodic_table import Element
from pymatgen.core.composition import Composition, CompositionError



class MaterialsTextProcessor:
    """
    Materials Science Text Processing Tools.
    """
    ELEMENTS = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K",
                "Ca", "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As", "Se", "Br", "Kr",
                "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd", "In", "Sn", "Sb", "Te", "I",
                "Xe", "Cs", "Ba", "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb",
                "Lu", "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", "Po", "At", "Rn", "Fr",
                "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr", "Rf",
                "Db", "Sg", "Bh", "Hs", "Mt", "Ds", "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og", "Uue"]

    ELEMENT_NAMES = ["hydrogen", "helium", "lithium", "beryllium", "boron", "carbon", "nitrogen", "oxygen", "fluorine",
                     "neon", "sodium", "magnesium", "aluminium", "silicon", "phosphorus", "sulfur", "chlorine", "argon",
                     "potassium", "calcium", "scandium", "titanium", "vanadium", "chromium", "manganese", "iron",
                     "cobalt", "nickel", "copper", "zinc", "gallium", "germanium", "arsenic", "selenium", "bromine",
                     "krypton", "rubidium", "strontium", "yttrium", "zirconium", "niobium", "molybdenum", "technetium",
                     "ruthenium", "rhodium", "palladium", "silver", "cadmium", "indium", "tin", "antimony", "tellurium",
                     "iodine", "xenon", "cesium", "barium", "lanthanum", "cerium", "praseodymium", "neodymium",
                     "promethium", "samarium", "europium", "gadolinium", "terbium", "dysprosium", "holmium", "erbium",
                     "thulium", "ytterbium", "lutetium", "hafnium", "tantalum", "tungsten", "rhenium", "osmium",
                     "iridium", "platinum", "gold", "mercury", "thallium", "lead", "bismuth", "polonium", "astatine",
                     "radon", "francium", "radium", "actinium", "thorium", "protactinium", "uranium", "neptunium",
                     "plutonium", "americium", "curium", "berkelium", "californium", "einsteinium", "fermium",
                     "mendelevium", "nobelium", "lawrencium", "rutherfordium", "dubnium", "seaborgium", "bohrium",
                     "hassium", "meitnerium", "darmstadtium", "roentgenium", "copernicium", "nihonium", "flerovium",
                     "moscovium", "livermorium", "tennessine", "oganesson", "ununennium"]

    ELEMENTS_AND_NAMES = ELEMENTS + ELEMENT_NAMES + [en.capitalize() for en in ELEMENT_NAMES]
    ELEMENTS_NAMES_UL = ELEMENT_NAMES + [en.capitalize() for en in ELEMENT_NAMES]

    # Elemement with the valence state in parenthesis.
    ELEMENT_VALENCE_IN_PAR = regex.compile(r"^("+r"|".join(ELEMENTS_AND_NAMES) +
                                           r")(\(([IV|iv]|[Vv]?[Ii]{0,3})\))$")
    ELEMENT_DIRECTION_IN_PAR = regex.compile(r"^(" + r"|".join(ELEMENTS_AND_NAMES) + r")(\(\d\d\d\d?\))")

    # Exactly IV, VI or has 2 consecutive II, or roman in parenthesis: is not a simple formula.
    VALENCE_INFO = regex.compile(r"(II+|^IV$|^VI$|\(IV\)|\(V?I{0,3}\))")

    SPLIT_UNITS = {"K", "h", "V", "wt", "wt.", "MHz", "kHz", "GHz", "Hz", "days", "weeks",
                   "hours", "minutes", "seconds", "T", "MPa", "GPa", "kPa", "at.", "mol.",
                   "at", "m", "N", "s-1", "vol.", "vol", "eV", "A", "atm", "bar",
                   "kOe", "Oe", "h.", "mWcm−2", "keV", "MeV", "meV", "day", "week", "hour",
                   "minute", "month", "months", "year", "cycles", "years", "fs", "ns",
                   "ps", "rpm", "g", "mg", "mAcm−2", "mA", "mK", "mT", "s-1", "dB",
                   "Ag-1", "mAg-1", "mAg−1", "mAg", "mAh", "mAhg−1", "m-2", "mJ", "kJ",
                   "m2g−1", "THz", "KHz", "kJmol−1", "Torr", "gL-1", "Vcm−1", "mVs−1",
                   "J", "GJ", "mTorr", "bar", "cm2", "mbar", "kbar", "mmol", "mol", "molL−1",
                   "MΩ", "Ω", "kΩ", "mΩ", "mgL−1", "moldm−3", "m2", "m3", "cm-1", "cm",
                   "Scm−1", "Acm−1", "eV−1cm−2", "cm-2", "sccm", "cm−2eV−1", "cm−3eV−1",
                   "kA", "s−1", "emu", "L", "cmHz1", "gmol−1", "kVcm−1", "MPam1",
                   "cm2V−1s−1", "Acm−2", "cm−2s−1", "MV", "ionscm−2", "Jcm−2", "ncm−2",
                   "Jcm−2", "Wcm−2", "GWcm−2", "Acm−2K−2", "gcm−3", "cm3g−1", "mgl−1",
                   "mgml−1", "mgcm−2", "mΩcm", "cm−2s−1", "cm−2", "ions", "moll−1",
                   "nmol", "psi", "mol·L−1", "Jkg−1K−1", "km", "nm", "cm", "Wm−2", "mass", "mmHg",
                   "mmmin−1", "GeV", "m−2", "m−2s−1", "Kmin−1", "gL−1", "ng", "hr", "w", "mW", "φ", "μ", 'fv', "RMF", 
                   "mN", "kN", "Mrad", "rad", "arcsec", "Ag−1", "dpa", "cdm−2",
                   "cd", "mcd", "mHz", "m−3", "ppm", "phr", "mL", "ML", "mlmin−1", "MWm−2",
                   "Wm−1K−1", "Wm−1K−1", "kWh", "Wkg−1", "Jm−3", "m-3", "gl−1", "A−1",
                   "Ks−1", "mgdm−3", "mms−1", "ks", "appm", "ºC", "HV", "kDa", "Da", "kG",
                   "kGy", "MGy", "Gy", "mGy", "Gbps", "μB", "μL", "μF", "nF", "pF", "mF",
                   "A", "Å", "A˚", "μgL−1"}
    
    STOPWORDS = []

    NR_BASIC = regex.compile(r"^[+-]?\d*\.?\d+\(?\d*\)?+$", regex.DOTALL)
    NR_AND_UNIT = regex.compile(r"^([+-]?\d*\.?\d+\(?\d*\)?+)([\p{script=Latin}|Ω|μ]+.*)", regex.DOTALL)

    PUNCT = list(string.punctuation) + ["\"", "“", "”", "≥", "≤", "×"]

    def __init__(self):
        self.elem_name_dict = {en: es for en, es in zip(self.ELEMENT_NAMES, self.ELEMENTS)}
        

    def tokenize(self, text, split_oxidation=True, keep_sentences=True):
        """Converts a string to a list tokens (words) using a modified chemdataextractor tokenizer.

        Adds a few fixes for inorganic materials science, such as splitting common units from numbers
        and splitting the valence state.

        Args:
            text: input text as a string
            split_oxidation: if True, will split the oxidation state from the element, e.g. iron(II)
                will become iron (II), same with Fe(II), etc.
            keep_sentences: if False, will disregard the sentence structure and return tokens as a
                single list of strings. Otherwise returns a list of lists, each sentence separately.

        Returns:
            A list of strings if keep_sentence is False, otherwise a list of list of strings, which each
            list corresponding to a single sentence.
        """
        def split_token(token, so=split_oxidation):
            """Processes a single token, in case it needs to be split up.

            There are 2 cases when the token is split: A number with a common unit, or an
            element with a valence state.

            Args:
                token: The string to be processed.
                so: If True, split the oxidation (valence) string. Units are always split.

            Returns:
                A list of strings.
            """
            elem_with_valence = self.ELEMENT_VALENCE_IN_PAR.match(token) if so else None
            nr_unit = self.NR_AND_UNIT.match(token)
            if nr_unit is not None and nr_unit.group(2) in self.SPLIT_UNITS:
                # Splitting the unit from number, e.g. "5V" -> ["5", "V"].
                return [nr_unit.group(1), nr_unit.group(2)]
                # return [nr_unit.group(1), '<UNIT>']
            elif elem_with_valence is not None:
                # Splitting element from it"s valence state, e.g. "Fe(II)" -> ["Fe", "(II)"].
                # return [elem_with_valence.group(1), elem_with_valence.group(2)]
                return [elem_with_valence.group(1)]
            else:
                return [token]

        cde_p = Paragraph(text)
        tokens = cde_p.tokens
        toks = []
        for sentence in tokens:
            if keep_sentences:
                toks.append([])
                for tok in sentence:
                    toks[-1] += split_token(tok.text, so=split_oxidation)
            else:
                for tok in sentence:
                    toks += split_token(tok.text, so=split_oxidation)
        return toks

    def process(self, tokens, exclude_punct=False, convert_num=True, normalize_materials=True, remove_accents=True,
                split_oxidation=True):
        """Processes a pre-tokenized list of strings or a string.

        Selective lower casing, material normalization, etc.

        Args:
            tokens: A list of strings or a string. If a string is supplied, will use the
                tokenize method first to split it into a list of token strings.
            exclude_punct: Bool flag to exclude all punctuation.
            convert_num: Bool flag to convert numbers (selectively) to <nUm>.
            normalize_materials: Bool flag to normalize all simple material formula.
            remove_accents: Bool flag to remove accents, e.g. Néel -> Neel.
            make_phrases: Bool flag to convert single tokens to common materials science phrases.
            split_oxidation: Only used if string is supplied, see docstring for tokenize method.

        Returns:
            A (processed_tokens, material_list) tuple. processed_tokens is a list of strings,
            whereas material_list is a list of (original_material_string, normalized_material_string)
            tuples.
        """
        
        if not isinstance(tokens, list):  # If it"s a string.
            return self.process(self.tokenize(
                tokens, split_oxidation=split_oxidation, keep_sentences=False),
                exclude_punct=exclude_punct,
                convert_num=convert_num,
                normalize_materials=normalize_materials,
                remove_accents=remove_accents,
                
            )

        processed, mat_list = [], []

        for i, tok in enumerate(tokens):
            if exclude_punct and tok in self.PUNCT:  # Punctuation.
                continue
            if len(tok) > 1 and (tok[-1] == '_' or tok[-1] == '-' or tok[-1] == '~' or tok[-1] == '^'): 
                # print('before:', tok)
                tok = tok[:-1]
                # print('after:', tok)
            if len(tok) > 1 and (tok[0] == '_' or tok[0] == '-' or tok[0] == '~' or tok[0] == '^'):
                # print('before:', tok)
                tok = tok[1:]
                # print('after:', tok)
            elif tok in self.SPLIT_UNITS:
                tok = '<UNIT>'
            elif convert_num and self.is_number(tok):  # Number.
                # Replace all numbers with <nUm>, except if it is a crystal direction (e.g. "(111)").
                try:
                    if tokens[i - 1] == "(" and tokens[i + 1] == ")" \
                            or tokens[i - 1] == "〈" and tokens[i + 1] == "〉":
                        pass
                    else:
                        tok = "<NUM>"
                except IndexError:
                    tok = "<NUM>"
            
            elif tok in self.ELEMENTS_NAMES_UL:  # Chemical element name.
                # Add as a material mention.
                mat_list.append((tok, self.elem_name_dict[tok.lower()]))
                tok = tok.lower()
            elif self.is_simple_formula(tok):  # Simple chemical formula.
                normalized_formula = self.normalized_formula(tok)
                mat_list.append((tok, normalized_formula))
                if normalize_materials:
                    tok = normalized_formula
            elif (len(tok) == 1 or (len(tok) > 1 and tok[0].isupper() and tok[1:].islower())) \
                    and tok not in self.ELEMENTS and tok not in self.SPLIT_UNITS \
                    and self.ELEMENT_DIRECTION_IN_PAR.match(tok) is None:
                # To lowercase if only first letter is uppercase (chemical elements already covered above).
                tok = tok.lower()

            if remove_accents:
                tok = self.remove_accent(tok)

            processed.append(tok)

        

        return processed, mat_list

    

    def is_number(self, s):
        """Determines if the supplied string is number.

        Args:
            s: The input string.

        Returns:
            True if the supplied string is a number (both . and , are acceptable), False otherwise.
        """
        return self.NR_BASIC.match(s.replace(",", "")) is not None

    @staticmethod
    def is_element(txt):
        """Checks if the string is a chemical symbol.

        Args:
            txt: The input string.

        Returns:
            True if the string is a chemical symbol, e.g. Hg, Fe, V, etc. False otherwise.
        """
        try:
            Element(txt)
            return True
        except ValueError:
            return False

    def is_simple_formula(self, text):
        """Determines if the string is a simple chemical formula.

        Excludes some roman numbers, e.g. IV.

        Args:
            text: The input string.

        Returns:
            True if the supplied string a simple formula, e.g. IrMn, LiFePO4, etc. More complex
            formula such as LiFePxO4-x are not considered to be simple formulae.
        """
        if self.VALENCE_INFO.search(text) is not None:
            # 2 consecutive II, IV or VI should not be parsed as formula.
            # Related to valence state, so don"t want to mix with I and V elements.
            return False
        elif any(char.isdigit() or char.islower() for char in text):
            # Aas to contain at least one lowercase letter or at least one number (to ignore abbreviations).
            # Also ignores some materials like BN, but these are few and usually written in the same way,
            # so normalization won"t be crucial.
            try:
                if text in ["O2", "N2", "Cl2", "F2", "H2"]:
                    # Including chemical elements that are diatomic at room temperature and atm pressure,
                    # despite them having only a single element.
                    return True
                composition = Composition(text)
                # Has to contain more than one element, single elements are handled differently.
                if len(composition.keys()) < 2 or any([not self.is_element(key) for key in composition.keys()]):
                    return False
                return True
            except (CompositionError, ValueError, OverflowError):
                return False
        else:
            return False

    @staticmethod
    def get_ordered_integer_formula(el_amt, max_denominator=1000):
        """Converts a mapping of {element: stoichiometric value} to a alphabetically ordered string.

        Given a dictionary of {element : stoichiometric value, ..}, returns a string with
        elements ordered alphabetically and stoichiometric values normalized to smallest common
        integer denominator.

        Args:
            el_amt: {element: stoichiometric value} mapping.
            max_denominator: The maximum common denominator of stoichiometric values to use for
                normalization. Smaller stoichiometric fractions will be converted to the same
                integer stoichiometry.

        Returns:
            A material formula string with elements ordered alphabetically and the stoichiometry
            normalized to the smallest integer fractions.
        """
        g = gcd_float(list(el_amt.values()), 1 / max_denominator)
        d = {k: round(v / g) for k, v in el_amt.items()}
        formula = ""
        for k in sorted(d):
            if d[k] > 1:
                formula += k + str(d[k])
            elif d[k] != 0:
                formula += k
        return formula

    def normalized_formula(self, formula, max_denominator=1000):
        """Normalizes chemical formula to smallest common integer denominator, and orders elements alphabetically.

        Args:
            formula: the string formula.
            max_denominator: highest precision for the denominator (1000 by default).

        Returns:
            A normalized formula string, e.g. Ni0.5Fe0.5 -> FeNi.
        """
        try:
            formula_dict = Composition(formula).get_el_amt_dict()
            return self.get_ordered_integer_formula(formula_dict, max_denominator)
        except (CompositionError, ValueError):
            return formula

    @staticmethod
    def remove_accent(txt):
        """Removes accents from a string.

        Args:
            txt: The input string.

        Returns:
            The de-accented string.
        """
        # There is a problem with angstrom sometimes, so ignoring length 1 strings.
        return unidecode.unidecode(txt) if len(txt) > 1 else txt


In [17]:
processor = MaterialsTextProcessor()

year_tokens = {}

for year, paper_lst in doi_absts.items():
    index = 0
    print(year)
    for paper in paper_lst:
        doi, title, abst = paper
        if index % 50 == 0: print(index, end = ' ')
        index += 1
        # print(type(abst))
        # break
        tokens, chemicals = processor.process(str(abst))

        if year not in year_tokens: year_tokens[year] = [(doi, title, tokens)]
        else: year_tokens[year].append((doi, title, tokens))
        
    print()

print(len(year_tokens))


2015
0 50 100 150 200 250 300 350 400 450 
2002
0 
2020
0 50 100 150 200 250 300 350 400 450 
2011
0 50 100 150 
2008
0 50 
2017
0 50 100 150 200 250 300 350 400 
2018
0 50 100 150 200 250 300 350 
2013
0 50 100 150 200 250 300 
2021
0 50 100 150 
2010
0 50 100 
2014
0 50 100 150 200 250 300 350 400 
2019
0 50 100 150 200 250 300 350 

0 
2012
0 50 100 150 200 250 
2016
0 50 100 150 200 250 300 350 400 450 
2001
0 
2004
0 
2009
0 50 
1997
0 
2005
0 
2007
0 
2003
0 
1992
0 
2006
0 
1996
0 
1962
0 
1998
0 
1971
0 
1995
0 
1965
0 
1956
0 
1994
0 
1999
0 
1970
0 
1964
0 
2000
0 
1991
0 
1983
0 
1961
0 
1906
0 
1922
0 
1986
0 
1988
0 
1985
0 
1959
0 
1963
0 
46


In [ ]:
# import sys
# sys.path.append('..')
# from mat2vec.processing import MaterialsTextProcessor

processor = MaterialsTextProcessor()

year_tokens = {}

for year, paper_lst in year_papers.items():
    index = 0
    print(year)
    for paper in paper_lst:
        doi, title, abst = paper
        if index % 500 == 0: print(index, end = ' ')
        index += 1
        # print(type(abst))
        # break
        tokens, chemicals = processor.process(str(abst))

        if year not in year_tokens: year_tokens[year] = [(doi, title, tokens)]
        else: year_tokens[year].append((doi, title, tokens))
        
    print()

print(len(year_tokens))


In [ ]:
print(year_tokens['2018'])

In [ ]:
# NOTE: 将 abstract 转为 tokens
import spacy, re

nlp = spacy.load('en_core_web_sm')
all_tokens = {}
# freq = {} # 计算每个词出现的频率
# for doi, (abst, title, year) in doi_absts.items():
for year, paper_lst in year_papers.items():
    # print(year_papers.items())
    
    index = 0
    print()
    print(year)
    for paper in paper_lst:
        doi, title, abst = paper
    # for doi, title, abst in paper_lst:
        # print(index)
        if index % 500 == 0: print(index, end = ' ')
        index += 1

        doc = nlp(str(abst))
        # print(doc)
        tokens = []
        for token in doc:
            # print(token)
            if re.match(r'(x|X)+(- | _)', token.shape_):
                print(token, end = ' ')
                token = nlp(str(token)[:-1])[0]
                print(token)
            if token.pos_ != 'PUNCT' and token.pos_ != 'SPACE' and not token.is_stop:
                if token.pos_ == 'NUM':
                    tokens.append('#NUM')
                    # if '#NUM' not in freq:
                    #     freq['#NUM'] = 1
                    # else: freq['#NUM'] += 1
                else: 
                    tokens.append(token.lemma_)
                    # if token.lemma_ not in freq: freq[token.lemma_] = 1
                    # else: freq[token.lemma_] += 1

        if year not in all_tokens: all_tokens[year] = [(doi, title, tokens)]
        else: all_tokens[year].append((doi, title, tokens))
    

print(len(all_tokens))
# TODO 去掉频率很低的词(目前论文数量太少，先不弄)

In [18]:
# NOTE: 初始化 db_tokens
import pymongo
client = pymongo.MongoClient('mongodb://localhost:27017')
db_tokens = client['year_tokens']
def tokens_init(db_tokens):
    # for col in db_tokens.list_collection_names():
    #     de = db_tokens[col]
    #     db_tokens.drop_collection(de)

    # for year in range(2000, 2022):
    for year in years:
        if year == '': year = 'None'
        col = db_tokens[str(year)]
        if year == 'None': year = ''
        for i in all_tokens[str(year)]:
            doi, title, tokens = i
            paper = {
                'doi': doi,
                'title': title,
                'tokens': tokens
            }
            col.insert_one(paper)
tokens_init(db_tokens)


NameError: name 'all_tokens' is not defined

In [20]:
# NOTE: 初始化 db_tokens_punct(带标点)
import pymongo
client = pymongo.MongoClient('mongodb://localhost:27017')
db_tokens_punct = client['year_tokens_punct']
def tokens_init(db_tokens_punct):
    # for col in db_tokens_punct.list_collection_names():
    #     de = db_tokens_punct[col]
    #     db_tokens_punct.drop_collection(de)

    # for year in range(2000, 2022):
    for year in years:
        if year == '': year = "None"
        col = db_tokens_punct[str(year)]
        if year == 'None': year = ''
        for i in year_tokens[str(year)]:
            doi, title, tokens = i
            paper = {
                'doi': doi,
                'title': title,
                'tokens': tokens
            }
            col.insert_one(paper)
tokens_init(db_tokens_punct)
print(1)

1


In [13]:
# NOTE: 初始化 db_tokens_punct_test(带标点)
import pymongo
client = pymongo.MongoClient('mongodb://localhost:27017')
db_tokens_punct = client['year_tokens_punct_test']
def tokens_init(db_tokens_punct):
    for col in db_tokens_punct.list_collection_names():
        de = db_tokens_punct[col]
        db_tokens_punct.drop_collection(de)

    for year in range(2000, 2022):
        col = db_tokens_punct[str(year)]
        for i in year_tokens[str(year)]:
            doi, title, tokens = i
            paper = {
                'doi': doi,
                'title': title,
                'tokens': tokens
            }
            col.insert_one(paper)
tokens_init(db_tokens_punct)
print(1)

1


In [ ]:
print(year_chemicals)
for year in range(2000, 2022):
    result = set()
    for tup in year_chemicals[str(year)]:
        for i in tup:
            result.add(i)
    year_chemicals[str(year)] = result
print(year_chemicals)


In [18]:
# NOTE: 初始化 db_chemicals
import pymongo
client = pymongo.MongoClient('mongodb://localhost:27017')
db_chemicals = client['year_chemicals']
def chemicals_init(db_chemicals):
    for col in db_chemicals.list_collection_names():
        de = db_chemicals[col]
        db_chemicals.drop_collection(de)

    for year in range(2000, 2022):
        col = db_chemicals[str(year)]
        
        item = {
            'chemical': list(year_chemicals[str(year)])
        }
        col.insert_one(item)

chemicals_init(db_chemicals)



In [2]:
# NOTE: 从数据库中取 all_tokens
all_tokens = {}
for year in db_tokens.list_collection_names():
    for paper in db_tokens[year].find({}, {'_id': 0}):
        if year not in all_tokens: all_tokens[year] = [(paper['doi'], paper['title'], paper['tokens'])]
        else: all_tokens[year].append((paper['doi'], paper['title'], paper['tokens']))

# print(all_tokens)

In [9]:
# NOTE: 从数据库中取 all_tokens_punct
all_tokens = {}
for year in db_tokens_punct.list_collection_names():
    for paper in db_tokens_punct[year].find({}, {'_id': 0}):
        if year not in all_tokens: all_tokens[year] = [(paper['doi'], paper['title'], paper['tokens'])]
        else: all_tokens[year].append((paper['doi'], paper['title'], paper['tokens']))

# print(all_tokens)

In [ ]:
print(all_tokens_punct)

In [11]:
# NOTE: 建立语料库
from gensim import utils

class Corpus:
    def __iter__(self):
        for year in all_tokens:
            token_lst = all_tokens[year]
            for tup in token_lst:
                tokens = tup[-1]
                yield tokens

corpus = Corpus()


In [ ]:
# NOTE: Word2Vec
import gensim.models

# for i in corpus: print(i)
w2v_model = gensim.models.Word2Vec(corpus)

In [ ]:
# NOTE: FastText
from gensim.models.fasttext import FastText

f_model = FastText()
f_model.build_vocab(corpus)

f_model.train(corpus_iterable=corpus, epochs=f_model.epochs, total_examples=f_model.corpus_count, total_words=f_model.corpus_total_words)
print(f_model)

In [ ]:
# NOTE: To store models
import tempfile
def store_w2v():
    with tempfile.NamedTemporaryFile(prefix='gensim-model-', delete=False) as tmp:
        w2v_path = tmp.name
        with open('w2v_model_path', 'w') as f:
            f.write(w2v_path)
        w2v_model.save(w2v_path)

def store_f():
    with tempfile.NamedTemporaryFile(prefix='gensim-model-', delete=False) as tmp:
        f_path = tmp.name
        with open('f_model_path', 'w') as f:
            f.write(f_path)
        f_model.save(f_path)

# store_w2v()
# store_f()

In [16]:
# NOTE: To load models
def load_w2v():
    with open('w2v_model_path') as f:
        w2v_path = f.read()
    return gensim.models.Word2Vec.load(w2v_path)

def load_f():
    with open('f_model_path') as f:
        f_path = f.read()
    return gensim.models.FastText.load(f_path)

w2v_model = load_w2v()
# f_model = load_f()
# print(w2v_path, f_path)

<IPython.core.display.Javascript object>

In [25]:
w2v_model.wv.most_similar('good')

[('excellent', 0.8196513056755066),
 ('well', 0.8011565804481506),
 ('reasonable', 0.7711325287818909),
 ('superior', 0.7699888944625854),
 ('satisfactory', 0.763913094997406),
 ('improved', 0.7163109183311462),
 ('outstanding', 0.6705237030982971),
 ('acceptable', 0.65166836977005),
 ('ideal', 0.5965924859046936),
 ('comparable', 0.5880300998687744)]

In [ ]:
f_model.wv.most_similar(('good'))

In [ ]:
w2v_model.wv.doesnt_match(['manganese', 'zinc', 'good', 'carbon'])

In [ ]:
w2v_model.wv.most_similar(positive=['bad', 'excellent'], negative=['good'])

In [23]:
def analogy(x1, x2, y1):
    try:
        result = w2v_model.wv.most_similar(positive=[y1, x2], negative=[x1])
        return result[0][0]
    except KeyError:
        return

In [26]:
analogy('good', 'excellent', 'bad')

'inferior'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA

def display_pca_scatterplot(model, words=None, sample=0):
    if words == None:
        if sample > 0:
            words = np.random.choice(list(model.vocab.keys()), sample)
        else:
            words = [ word for word in model.vocab ]
        
    word_vectors = np.array([model[w] for w in words])

    twodim = PCA().fit_transform(word_vectors)[:,:2]
    
    plt.figure(figsize=(6,6))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)

In [ ]:
display_pca_scatterplot(w2v_model.wv, ['China', 'England', 'long', 'short', 'increase', 'metal', 'America', 'iron', 'carbon', 'fire', 'Mg', 'zinc', 'magnetic', 'decrease', 'nickel', 'good', 'excellent', 'bad', 'manganese'])

In [ ]:
from gensim import corpora

dic = []
all_texts = []
for year in all_tokens:
    token_lst = all_tokens[year]
    for doi, title, tokens in token_lst:
        all_texts += tokens
        dic.append(tokens)
    
# print(all_texts)
dic = corpora.Dictionary(dic)
print(len(all_texts))
print(len(dic))
# print(dic.token2id)


In [ ]:
# 化学品名称数量
from chemdataextractor.doc import Document

doc = ''
# print(all_texts)
for text in all_texts[:20000]:
    # print(text)
    doc += text + ' '
doc = Document(doc)
print(len(doc.records.serialize()))
# print(doc.records.serialize())



In [24]:
with open('analogies.txt') as f:
    lines = f.read().strip().split('\n')

pos = 0
neg = 0
for line in lines:
    if line[0] == ':': continue
    words = line.strip().split(' ')
    result = analogy(words[0], words[1], words[2])
    if result is not None and result == words[3]: pos += 1
    else: neg += 1

print(pos, neg)



1705 33399
